In [7]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import scipy.io as sio
import sys
sys.path.append('./helpers')
import loadmat as lm
from sklearn import linear_model
from scipy import signal
from sklearn.model_selection import cross_val_score, cross_validate
import os
from sklearn.metrics import make_scorer, confusion_matrix
import glob
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from helpers import preprocess

In [6]:
files = glob.glob('Z:/giocomo/attialex/NP_DATA/*_gain_*.mat')
dataset = lm.loadmat(files[1])
dataset_p = preprocess(dataset,5)


TypeError: preprocess() takes 1 positional argument but 2 were given

In [13]:
dataset_p = preprocess(dataset)
dataset_p['spikecount']
model = linear_model.LinearRegression()
model.fit(dataset_p['spikecount'][trialidx,:],dataset_p['posx_bin'][trialidx])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
bl_trials = np.nonzero(np.all([dataset_p['trial_contrast']==100, dataset_p['trial_gain']==1],axis = 0))
trialidx = np.isin(dataset_p['trial_resampled'],bl_trials)